#Análise Comparativa

##1. Preparação dos dados

###1.1 Configurações Iniciais

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import Markdown
from sklearn.pipeline import TransformerMixin # Usada no tratamento de outliers
from scipy.spatial.distance import cdist # Usada no tratamento de outliers
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold# Para a validação cruzada

# Para os modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score# Para métricas

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler# Para pré-processamento
import matplotlib.pyplot as plt

###1.2 Carregando os dados

In [6]:
df = df = pd.read_csv(r'https://raw.githubusercontent.com/lrodriguesg/bike-bytes/master/data/raw/data.csv')
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [ ]:
# Dicionário dos dados
data_path = ('https://raw.githubusercontent.com/lrodriguesg/bike-bytes/master/data/external/dicionario.csv')
df_dict = pd.read_csv(data_path, sep=',')
pd.set_option('display.max_colwidth', None)

df_dict

,variavel,significado,tipo,subtipo,resposta
0,instant,Índice de cada registro,Quantitativa,Discreta,True
1,dteday,Data,Quantitativa,Data,False
2,season,"Estação do ano (1:inverno, 2:primavera, 3:verão, 4:outono)",Qualitativa,Nominal,False
3,yr,"Ano em que ocorreu a locação (0: 2011, 1: 2012)",Quantitativa,Discreta,False
4,mnth,Mês,Qualitativa,Ordinal,False
5,holiday,"Indica se era ou não feriado (0: não, 1: sim)",Qualitativa,Nominal,False
6,weekday,Dia da semana,Qualitativa,Ordinal,False
7,workingday,"Indica se é dia útil ou não (0: não, 1: sim)",Qualitativa,Nominal,False
8,weathersit,"Condição climática (1: Limpo, 2: Nublado, 3: Leve neve/chuva, 4: Chuva forte/neve)",Qualitativa,Ordinal,False
9,temp,"Temperatura normalizada em Celsius(min = -8 graus, max = 39 graus)",Quantitativa,Contínua,False


###1.3 Tratamento dos dados

In [7]:
# Seleção de Variáveis
df_X = df[['season', 'temp', 'atemp', 'mnth', 'weekday', 'workingday', 'weathersit', 'hum', 'windspeed']] #independente
df_y = df['cnt']# Depndente

In [8]:
#Dados faltantes
display(Markdown("#### Quantidade faltante "))
display(df_X.isnull().sum())

display(Markdown("#### Quantidade de linhas e linhas"))
display(df_X.shape)

#### Quantidade faltante 

season        0
temp          0
atemp         0
mnth          0
weekday       0
workingday    0
weathersit    0
hum           0
windspeed     0
dtype: int64

#### Quantidade de linhas e linhas

(731, 9)

In [9]:
display(Markdown("#### Quantidade faltante "))
display(df_y.isnull().sum())

display(Markdown("#### Quantidade de linhas e linhas"))
display(df_y.shape)

#### Quantidade faltante 

0

#### Quantidade de linhas e linhas

(731,)

In [10]:
class OutlierExtractor(TransformerMixin):
    def __init__(self, threshold=3, **kwargs):
        self.threshold = threshold
        self.kwargs = kwargs

    def transform(self, X, y=None):
        X = np.asarray(X)
        std = np.std(X)
        outliers_zscore = np.abs(cdist(X.mean(axis=0)[np.newaxis], X) / std)[0]
        if y is not None:
            y = np.asarray(y)
            return (
                X[outliers_zscore >= self.threshold, :],
                y[outliers_zscore >= self.threshold]
            )
        return X[outliers_zscore >= self.threshold, :]

    def fit(self, *args, **kwargs):
        return self


In [11]:
outlier_transformer = OutlierExtractor(threshold=3)
df_X, df_y = outlier_transformer.transform(df_X, y=df_y)

##2.Escolha do Modelo

###2.1 Metodologia

Neste estudo, o nosso objetivo principal é prever o número diário de aluguéis de bicicletas. Para atingir esse objetivo, focamos na análise de variáveis específicas que mostraram ter uma correlação significativa com os aluguéis em uma análise anterior. As variáveis em questão são: **sensação térmica (atemp)**, **temp(temperatura)** e a **estação do ano (season)**. Acreditamos que essas variáveis as mais relevantes para entender os padrões de locação e, portanto, serão essenciais para nossos modelos de aprendizado de máquina ao fazer previsões precisas sobre a demanda futura.

Os modelos de machine learning que exploraremos neste estudo incluem:

* **Regressão Linear (RL)**: Nossa baseline, escolhido como nossa baseline devido à sua natureza interpretativa.
* **K-Nearest Neighbors (KNN)**
* **Support Vector Regressor (SVR)**:
* **Árvore de Decisão para Regressão**

Avaliaremos os modelos com base nas seguintes métricas:

* **MSE (Erro Quadrático Médio)**
* **RMSE (Raiz do Erro Quadrático Médio)**
* **R² (Coeficiente de Determinação)**

Para garantir a avaliação dos nossos modelos, adotamos a estratégia de validação cruzada k-fold. A validação cruzada k-fold é benéfica pois nos permite maximizar tanto o treinamento quanto o teste dos dados, garantindo uma avaliação mais abrangente do desempenho do modelo. Além disso, ao treinar o modelo em diferentes subconjuntos, podemos ter mais confiança na sua capacidade de generalização e menos propensos a overfitting.

Nesta seção, definimos os modelos que serão avaliados, juntamente com suas respectivas configurações e parâmetros a serem considerados na busca em grid

Estes modelos e parâmetros nos fornecerão uma variedade de combinações a serem avaliadas, garantindo uma busca para encontrar o melhor ajuste para nosso conjunto de dados.


In [12]:
models = {
    "Regressão Linear": LinearRegression(),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(max_iter=10000),
    "Árvore de Decisão": DecisionTreeRegressor()
}

###2.2 Configurando o Experimento


In [20]:
# Divisão dos Dados
X = df[['season', 'temp', 'atemp', 'mnth', 'weekday', 'workingday', 'weathersit', 'hum', 'windspeed']]
y = df[['cnt']]

# Aplicar o One-Hot Encoding nas colunas categóricas
encoder = OneHotEncoder(sparse_output=False, drop='first')
X_encoded = encoder.fit_transform(X[['season', 'mnth', 'weekday', 'workingday', 'weathersit']])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['season', 'mnth', 'weekday', 'workingday', 'weathersit']))
X = pd.concat([X.drop(['season', 'mnth', 'weekday', 'workingday', 'weathersit'], axis=1), X_encoded_df], axis=1)



# Modelos
modelos = [
    ('Regressão Linear', LinearRegression()),
    ('KNN', KNeighborsRegressor(n_neighbors=5)),
    ('SVR', SVR(kernel='linear')),
    ('Árvore de Decisão', DecisionTreeRegressor())
]

# Validação Cruzada K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
resultados_cross2 = []

for nome_modelo, modelo in modelos:
    rmse_scores = []
    mse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Ajuste o scaler
        scaler = MinMaxScaler()
        y_train = scaler.fit_transform(y_train)

        # Transforme y_test usando o scaler ajustado
        y_test = scaler.transform(y_test)

        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        # Inverter a normalização para obter os valores reais
        y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
        y_test = scaler.inverse_transform(y_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        rmse_scores.append(rmse)
        mse_scores.append(mse)
        r2_scores.append(r2)

    media_rmse = np.mean(rmse_scores)
    media_mse = np.mean(mse_scores)
    media_r2 = np.mean(r2_scores)

    resultados_cross2.append([nome_modelo, media_rmse, media_mse, media_r2])

# Criar DataFrame com os resultados
df_resultados_cross_val = pd.DataFrame(resultados_cross2, columns=['Modelo', 'Média RMSE', 'Média MSE', 'Média R²'])

df_resultados_cross_val


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

,Modelo,Média RMSE,Média MSE,Média R²
0,Regressão Linear,1311.060941,1.720848e+06,0.531764
1,KNN,1566.260103,2.453673e+06,0.332462
2,SVR,1305.795919,1.706476e+06,0.535448
3,Árvore de Decisão,1735.710746,3.022383e+06,0.183866


###2.3 Resultados

In [ ]:
# Criando os subplots 2x2
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análise dos Modelos', fontsize=16)

# Primeiro gráfico: Média RMSE
axes[0, 0].barh(df_resultados_cross_val['Modelo'], df_resultados_cross_val['Média RMSE'])
axes[0, 0].set_title('Média RMSE')
axes[0, 0].set_xlabel('RMSE')
axes[0, 0].invert_yaxis()

# Segundo gráfico: Média MSE
axes[0, 1].barh(df_resultados_cross_val['Modelo'], df_resultados_cross_val['Média MSE'])
axes[0, 1].set_title('Média MSE')
axes[0, 1].set_xlabel('MSE')
axes[0, 1].invert_yaxis()

# Terceiro gráfico: Média R²
axes[1, 0].barh(df_resultados_cross_val['Modelo'], df_resultados_cross_val['Média R²'])
axes[1, 0].set_title('Média R²')
axes[1, 0].set_xlabel('R²')
axes[1, 0].invert_yaxis()

# O quarto gráfico (axes[1, 1]) pode ser usado para outra métrica ou pode ser escondido se desejar
axes[1, 1].axis('off')  # esconde o quarto gráfico

# Ajuste os gráficos
plt.tight_layout()
plt.subplots_adjust(top=0.9)

# Mostra os gráficos
plt.show()